# Terraform AWS Schema

#### DO THIS ONCE:

Generate the `aws-schema.json` with:

    `terraform providers schema -json > aws-schema.json`

In [ ]:
import json

In [ ]:
AWS_SCHEMA_PATH = 'aws-schema.json'
with open(AWS_SCHEMA_PATH) as f:
    schema = json.loads(f.read())

In [ ]:
aws = schema['provider_schemas']['registry.terraform.io/hashicorp/aws']

aws_resource_schemas = aws['resource_schemas']
aws_data_source_schemas = aws['data_source_schemas']

In [ ]:
from difflib import get_close_matches
import re

for elem_k, elem_v in aws_resource_schemas.items():
    print(elem_k)

    # get foreign keys without the 'aws' prefix
    fk_attrs = [re.sub(r'^aws_', '', fk) for fk in aws_resource_schemas.keys()]

    # Iterate block attributes
    elem_attrs = elem_v['block']['attributes']
    elem_attr_keys = [k for k, v in elem_attrs.items() 
                      if 'string' in v['type'] 
                      and k != elem_k
                      and not k.endswith('_state')
                      and not k.endswith('_type') ] 
    # alternatively, check only if ends with '_id', '_ids' or k itself

    # find possible references by closest match
    for k in elem_attr_keys:
        fk_attrs_for_k = get_close_matches(k, [fk for fk in fk_attrs if fk in k])
        if fk_attrs_for_k:
            #print(f'\t{k}')
            #print(f'\t\t{fk_attrs_for_k}')
            elem_attrs[k]['mm_type'] = f'aws_{fk_attrs_for_k[0]}'

    # Descent into blocks
    elem_blocktypes = elem_v['block'].get('block_types', {})
    for bt_k, bt_v in elem_blocktypes.items():
        print(f'\t{bt_k}')
        if bt_attrs := bt_v['block'].get('attributes'):
            # Iterate block attributes
            bt_attr_keys = [k for k, v in bt_attrs.items() 
                              if 'string' in v['type'] 
                              and k != elem_k
                              and not k.endswith('_state')
                              and not k.endswith('_type') ]
            print(f'\t\t{bt_attr_keys}')
            
            # find possible references by closest match
            for k in bt_attr_keys:
                fk_attrs_for_k = get_close_matches(k, [fk for fk in fk_attrs if fk in k])
                if fk_attrs_for_k:
                    print(f'\t\t{fk_attrs_for_k}')
                    bt_attrs[k]['mm_type'] = f'aws_{fk_attrs_for_k[0]}'

Search for `mm_type` in the output to view associated types.

> Should data_source_schemas be considered too?

In [ ]:
aws_resource_schemas

Now `aws_resource_schemas` has the `mm_type` on its attributes.

In [ ]:
import hcl2

AWS_TF_PATH = 'main.tf'
with open(AWS_TF_PATH) as f:
    tf = hcl2.load(f)

tf_resources = tf['resource']